In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [2]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=1, patch_size=7, emb_size=64, img_size=28):
        super().__init__()
        self.patch_size = patch_size
        self.projection = nn.Sequential(
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e')
        )
        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))
        self.pos_embedding = nn.Parameter(torch.randn((img_size // patch_size) ** 2 + 1, emb_size))

    def forward(self, x):
        b, _, _, _ = x.shape
        x = self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        x = torch.cat([cls_tokens, x], dim=1)
        x += self.pos_embedding
        return x


In [3]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, emb_size, num_heads):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.fc = nn.Linear(emb_size, emb_size)

    def forward(self, x):
        qkv = self.qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.num_heads), qkv)
        attn = torch.einsum('bhid,bhjd->bhij', q, k) / (self.emb_size ** (1/2))
        attn = attn.softmax(dim=-1)
        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.fc(out)

class TransformerBlock(nn.Module):
    def __init__(self, emb_size, num_heads, forward_expansion=4, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadSelfAttention(emb_size, num_heads)
        self.norm1 = nn.LayerNorm(emb_size)
        self.norm2 = nn.LayerNorm(emb_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(emb_size, forward_expansion * emb_size),
            nn.GELU(),
            nn.Linear(forward_expansion * emb_size, emb_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_out = self.attention(x)
        x = self.norm1(x + self.dropout(attn_out))
        ff_out = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x


In [4]:
class VisionTransformer(nn.Module):
    def __init__(self, in_channels=1, num_classes=10, emb_size=64, num_heads=4, depth=6, patch_size=7, img_size=28):
        super().__init__()
        self.patch_embedding = PatchEmbedding(in_channels, patch_size, emb_size, img_size)
        self.transformer = nn.Sequential(
            *[TransformerBlock(emb_size, num_heads) for _ in range(depth)]
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, num_classes)
        )

    def forward(self, x):
        x = self.patch_embedding(x)
        x = self.transformer(x)
        return self.mlp_head(x[:, 0])


In [5]:
def train_vit(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')

def test_vit(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss}, Accuracy: {accuracy * 100}%')


In [6]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    batch_size = 64
    lr = 0.001
    epochs = 10


    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    model = VisionTransformer().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()


    for epoch in range(1, epochs + 1):
        train_vit(model, device, train_loader, optimizer, criterion, epoch)
        test_vit(model, device, test_loader, criterion)


    torch.save(model.state_dict(), "vit_mnist_model.pth")
    print("Model saved as vit_mnist_model.pth")

if __name__ == '__main__':
    main()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:10<00:00, 939688.81it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 131824.41it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 244154.02it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3983799.41it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.3978350591887511
Test Loss: 0.0020725001292303206, Accuracy: 95.75%
Epoch 2, Loss: 0.13459080051003233
Test Loss: 0.00194347941307351, Accuracy: 96.03%
Epoch 3, Loss: 0.10372034123123709
Test Loss: 0.0012858115376671777, Accuracy: 97.43%
Epoch 4, Loss: 0.08513541262584931
Test Loss: 0.0014866813399945385, Accuracy: 97.11%
Epoch 5, Loss: 0.07552716850679415
Test Loss: 0.0014613713509519584, Accuracy: 97.08%
Epoch 6, Loss: 0.06752264775437063
Test Loss: 0.0010180691562592984, Accuracy: 97.99%
Epoch 7, Loss: 0.06294518103028761
Test Loss: 0.00104467077270383, Accuracy: 98.00999999999999%
Epoch 8, Loss: 0.05711004600191473
Test Loss: 0.0010373986833379604, Accuracy: 98.02%
Epoch 9, Loss: 0.05337616038654667
Test Loss: 0.0008836555976886303, Accuracy: 98.21%
Epoch 10, Loss: 0.047576574115718064
Test Loss: 0.000912204745481722, Accuracy: 98.17%
Model saved as vit_mnist_model.pth
